## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-14-14-51-51 +0000,nyt,"As U.S. Warships Patrol Caribbean, Drug Smuggl...",https://www.nytimes.com/2025/10/14/us/us-drug-...
1,2025-10-14-14-50-00 +0000,wsj,"Ford Cuts Production of Five Trucks, SUVs Afte...",https://www.wsj.com/business/autos/ford-plants...
2,2025-10-14-14-46-00 +0000,wsj,California Gov. Newsom Vetoes Bill That Would ...,https://www.wsj.com/articles/california-gov-ne...
3,2025-10-14-14-41-00 +0000,wsj,Maine’s Democratic Gov. Janet Mills officially...,https://www.wsj.com/politics/elections/susan-c...
4,2025-10-14-14-40-25 +0000,nyt,Suspect Pleads Guilty in Firebombing of Pa. Go...,https://www.nytimes.com/2025/10/14/us/arson-ha...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2323/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
122,trump,53
66,gaza,24
323,hostages,17
64,president,17
130,china,16


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
13,2025-10-14-14-24-22 +0000,nypost,Kamala Harris lauds ‘the president’ for Israel...,https://nypost.com/2025/10/14/us-news/kamala-h...,136
263,2025-10-13-20-45-44 +0000,nypost,Dems largely quiet about Trump’s role in histo...,https://nypost.com/2025/10/13/us-news/top-dems...,135
302,2025-10-13-17-58-10 +0000,nypost,Trump boasts Gaza peace process could be presi...,https://nypost.com/2025/10/13/us-news/trump-sa...,133
258,2025-10-13-20-53-37 +0000,wapo,Trump and world leaders meet in Egypt on futur...,https://www.washingtonpost.com/world/2025/10/1...,130
290,2025-10-13-18-40-17 +0000,nypost,Trump trolls Norway rep over Nobel Peace Prize...,https://nypost.com/2025/10/13/us-news/trump-tr...,126


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
13,136,2025-10-14-14-24-22 +0000,nypost,Kamala Harris lauds ‘the president’ for Israel...,https://nypost.com/2025/10/14/us-news/kamala-h...
290,58,2025-10-13-18-40-17 +0000,nypost,Trump trolls Norway rep over Nobel Peace Prize...,https://nypost.com/2025/10/13/us-news/trump-tr...
3,55,2025-10-14-14-41-00 +0000,wsj,Maine’s Democratic Gov. Janet Mills officially...,https://www.wsj.com/politics/elections/susan-c...
341,54,2025-10-13-14-55-52 +0000,nypost,JPMorgan to invest $10B in areas ‘critical’ to...,https://nypost.com/2025/10/13/business/jpmorga...
321,54,2025-10-13-16-51-39 +0000,bbc,"'You're my life, my hero': Hostages reunite wi...",https://www.bbc.com/news/articles/cyv8p8m4qg6o...
190,51,2025-10-14-01-59-18 +0000,cbc,"New York Times, AP, Reuters and Newsmax among ...",https://www.cbc.ca/news/world/new-york-times-a...
49,50,2025-10-14-13-00-00 +0000,wsj,"Tariffs, inflation and other threats such as e...",https://www.wsj.com/economy/trade-tensions-wei...
187,47,2025-10-14-02-41-46 +0000,nypost,"Ex-student allegedly stabs 3 staff members, pl...",https://nypost.com/2025/10/13/us-news/ex-stude...
234,43,2025-10-13-21-59-00 +0000,wsj,What has Trump done as president so far? The W...,https://www.wsj.com/politics/policy/donald-tru...
158,35,2025-10-14-05-51-43 +0000,cbc,"Dozens dead, missing, thousands of homes destr...",https://www.cbc.ca/news/world/mexico-floods-ra...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
